In [1]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

## Funciones

#### Funcion para descargar los datos

In [ ]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL_nits = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        # consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', CONCAT(CONCAT(VTANIT, '-'), VTASUC) AS 'Nitcliente-sucursal', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH BETWEEN '2021-01-01' AND '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1), CONCAT(CONCAT(VTANIT, '-'), VTASUC)"
        consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', CONCAT(CONCAT(VTANIT, '-'), VTASUC) AS 'Nitcliente-sucursal', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1), CONCAT(CONCAT(VTANIT, '-'), VTASUC)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL_nits = pd.concat([df_SQL_nits, df_SQL_int], ignore_index= True)

    df_SQL = df_SQL_nits.groupby("Fecha").sum().reset_index()
    df_SQL_nits = df_SQL_nits.groupby("Nitcliente-sucursal").sum().reset_index()

    return df_SQL, df_SQL_nits

#### Funcion EDA

In [ ]:
fig_kwargs = {
        # "renderer": "notebook",
        "renderer": "png",
        "width": 1000,
        "height": 600,}

def lineplot(bd):
        x = bd["Fecha"]
        x_n = np.arange(0, len(bd))
        y = bd["Ventas"]   
        coeficientes = np.polyfit(x_n, y, 1)
        poli = np.poly1d(coeficientes)

        trace1 = go.Scatter(x=x, y=y, mode='lines+markers', name='Ventas')
        trace2 = go.Scatter(x=x, y=poli(x_n), mode='lines', name='Línea de Tendencia')

        layout = go.Layout(
                title='Ventas por mes',
                xaxis=dict(title='Fecha'),
                yaxis=dict(title='Ventas'),
                legend=dict(x=1, y=1)
        )

        fig = go.Figure()
        fig.add_trace(trace1)
        fig.add_trace(trace2)
        fig.update_layout(layout)
        fig.show()

def plot_estacionalidad(bd):
        setup_ventas = setup(
                bd, #df
                target= "Ventas",
                ignore_features= "Suavizado",
                index= "Fecha",
                session_id = 42, #id para mantener replicabilidad
                transform_target= None, #transformador del target, "box-cox", "log", "sqrt", "exp", "cos"
                coverage= 0.9, #intervalos
                fh = 6,
                use_gpu= True,
                verbose= True,
                hyperparameter_split= 'all', # all or train
                )

        setup_ventas.plot_model(plot= "acf")
        setup_ventas.plot_model(plot= "pacf", data_kwargs= {"nlags": 12})

def estadisticas_estacionalidad(bd):
        setup_ventas = setup(
                bd, #df
                target= "Ventas",
                ignore_features= "Suavizado",
                index= "Fecha",
                session_id = 42, #id para mantener replicabilidad
                transform_target= None, #transformador del target, "box-cox", "log", "sqrt", "exp", "cos"
                coverage= 0.9, #intervalos
                fh = 6,
                use_gpu= True,
                verbose= True,
                hyperparameter_split= 'all', # all or train
                )
        #Prueba Dickey-Fuller
        df_adf = setup_ventas.check_stats(test='stationarity')
        df_adf = df_adf[df_adf["Test Name"] == "ADF"].iloc[:,-3:]
        p_value = round(df_adf["Value"][1], 4)
        df_adf2 = df_adf.iloc[2:,:]
        df_adf2 = df_adf2[["Property", "Value"]]

        print("Se realiza la prueba de Dickey-Fuller para analizar la estacionalidad de la serie de tiempo.")
        print("H0: la serie no es estacionaria")
        print("Con un alfa de 5% se determina:")
        #p_value
        if p_value < 0.05:
                print(f"Se rechaza la H0 ya que el p_valor de {p_value} es menor al alfa")
        else:
                print(f"No se tiene suficiente evidencia para rechazar la H0, un p_valor de {p_value} es mayor al alfa")
        print("Por otro lado, los valores criticos de la prueba son los siguientes:")
        print(df_adf2)

def EDA_cluster(bd, bd_nits):
        #Variables
        bd = bd.reset_index()
        bd_nits = bd_nits
        primer_fecha = datetime.utcfromtimestamp(bd.iloc[0, 1].timestamp())
        ultima_fecha = datetime.utcfromtimestamp(bd.iloc[-1, 1].timestamp())
        describe_bd = bd.describe().applymap("{:,.0f}".format)
        describe_bd_nits = bd_nits.describe().applymap("{:,.0f}".format)

        print(f"Esta base de datos tiene ventas de {len(bd)} meses,")
        print(f"empezando desde el {primer_fecha.strftime('%d-%m-%Y')}")
        print(f"y terminando el {ultima_fecha.strftime('%d-%m-%Y')}")
        print("La composicion estadistica de la base de datos es la siguiente:")
        print(describe_bd["Ventas"][1:])
        print(" ")

        print(f"Por otro lado, esta compuesto por ventas de {len(bd_nits)} clientes")
        print("Y asi se comporta estadisticamente asi:")
        print(describe_bd_nits["Ventas"][1:])
        print(" ")
        lineplot(bd)
        plot_estacionalidad(bd)
        estadisticas_estacionalidad(bd)

### Funcion suavizacion

In [ ]:
def plot_suavizacion(bd, alpha, tipo_suavizacion):
    # Visualizar la serie original y suavizada
    x = bd["Fecha"]
    y1 = bd["Ventas"]
    y2 = bd["Suavizado"]
    trace1 = go.Scatter(x= x, y= y1, mode= "lines+markers", name= "Ventas")
    trace2 = go.Scatter(x= x, y= y2, mode= "lines+markers", name= f"Suavizado alfa {alpha}")

    layout = go.Layout(
            title=f"Ventas por mes, suavizado con {tipo_suavizacion} y alpha {alpha}",
            xaxis=dict(title="Fecha"),
            yaxis=dict(title="Ventas"),
            legend=dict(x=1, y=1)
            )
    # fig = go.Figure()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1, secondary_y= False)
    fig.add_trace(trace2, secondary_y= True)
    fig.update_yaxes(title_text="Ventas", secondary_y=False)
    fig.update_yaxes(title_text="Suavizacion", secondary_y=True)
    fig.update_layout(layout)
    fig.show()

def suavizacion(bd, alpha= 0.2, tipo_suavizacion= "exponencial simple"):
    # Aplicar suavización exponencial simple
    if tipo_suavizacion == "exponencial simple":
        bd["Suavizado"] = bd["Ventas"].ewm(alpha=alpha).mean()
    elif tipo_suavizacion == "logaritmica":
        bd["Suavizado"] = np.log(bd["Ventas"]).ewm(alpha=alpha).mean()
    else:
        bd["Suavizado"] = np.sqrt(bd["Ventas"]).ewm(alpha=alpha).mean()
    
    plot_suavizacion(bd, alpha, tipo_suavizacion)
    
    return bd

def inversa_suavizacion(bd, tipo_suavizacion= "exponencial simple"):
    if tipo_suavizacion == "logaritmica":
        print(np.exp(bd))
    elif tipo_suavizacion == "raiz cuadrada":
        print(bd.apply(lambda x: x**2))
    else:
        print(bd)

## Extraccion, tranformacion y analisis

#### Carga de los clusters

In [ ]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

#El mejor modelo es 10-Institucional-A
filtro_distrito = 10
filtro_tipozona = "Institucional"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                            (df_clusters["Codigo distrito"] == filtro_distrito) &
                            (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                            (df_clusters["Letra cluster"] == filtro_cluster)
                            ]

#### EDA del cluster elegido

In [ ]:
# Definicion del df a realizar EDA
df_clusters_EDA = df_clusters.groupby(["Codigo distrito", "Nombre tipo zona", "Letra cluster"]).agg({"Nit cliente-sucursal": np.size}).reset_index()

In [ ]:
# Variables auxiliares
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [ ]:
#dfs para EDA y forecast
ventas_cluster, ventas_nits = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [ ]:
EDA_cluster(ventas_cluster, ventas_nits)

##### Pruebas

In [ ]:
# El tipo de suavizacion puede ser:
    #Exponencial simple
    #Logaritmica
    #Raiz cuadrada
ventas_cluster = suavizacion(ventas_cluster, alpha= 0.2, tipo_suavizacion= "exponencial simple")
# ventas_cluster = suavizacion(ventas_cluster, alpha= 0.2, tipo_suavizacion= "logaritmica")
# ventas_cluster = suavizacion(ventas_cluster, alpha= 0.2, tipo_suavizacion= "raiz cuadrada")

##### Configuracion y prediccion

In [ ]:
setup_suavizado = setup(
            ventas_cluster, #df
            target= "Suavizado",
            ignore_features= "Ventas",
            index= "Fecha",
            session_id = 42, #id para mantener replicabilidad
            transform_target= None, #transformador del target, "box-cox", "log", "sqrt", "exp", "cos"
            coverage= 0.9, #intervalos
            fh = 6,
            use_gpu= True,
            verbose= True
            )

In [ ]:
# remove_metric('RMSE')
# remove_metric('RMSSE')
# remove_metric('SMAPE')
# remove_metric('MAE')
top_models = compare_models(
                        n_select= 3,
                        sort= "r2"
                        )
metricas_completas = pull()

In [ ]:
# Establecer el formato de visualización de números en notación decimal
pd.options.display.float_format = '{:,.3f}'.format
metricas_completas.loc[metricas_completas["R2"] > 0]
# metricas_completas
# metricas_completas[:3]

### Prediction

In [ ]:
plot_model(top_models, plot = 'forecast', data_kwargs = {'fh': 9})

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
final_model = setup_suavizado.finalize_model(top_models[0])
inversa_suavizacion(setup_suavizado.predict_model(final_model), tipo_suavizacion= "exponencial simple")
setup_suavizado.plot_model(final_model)